In [6]:
import pandas as pd

# Data Preprocessing Internal Data

In [2]:
!pip install statsmodels pmdarima prophet

In [5]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA, ARIMA
# from pmdarima.arima import auto_arima
# from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
# from fbprophet import Prophet

from math import sqrt

import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import seaborn as sns

from random import random

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error


In [113]:
df = pd.read_csv('internal_data/data_707332.csv')

# df

# -6.983124, 110.409652

# https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2041-02-01&end_date=2025-06-01&daily="temperature_2m_max", "temperature_2m_min"

In [114]:
df.head()

,id,outlet_id,settled_at,name,sku,quantity,unit_name,price_sell_unit,payment_mode,total_discount,total_collected,total_tax,total_gross
0,601908266,740514,2024-02-29 19:13:12,MieNas,NaN,1.0,Pcs,20000.0,CASH,0.0,20000.0,0.0,20000.0
1,601908266,740514,2024-02-29 19:13:12,Nasi Goreng Oriental,NaN,1.0,Pcs,13000.0,CASH,0.0,13000.0,0.0,13000.0
2,601908266,740514,2024-02-29 19:13:12,Jeruk,NaN,1.0,Pcs,12000.0,CASH,0.0,12000.0,0.0,12000.0
3,601908266,740514,2024-02-29 19:13:12,Sop Buah,NaN,1.0,Pcs,17000.0,CASH,0.0,17000.0,0.0,17000.0
4,601907323,740514,2024-02-29 19:12:15,Ayam Penyet,NaN,3.0,Pcs,25000.0,CASH,0.0,75000.0,0.0,75000.0


In [115]:
df.shape

(21651048, 13)

In [116]:
df.describe()

,id,outlet_id,sku,quantity,price_sell_unit,total_discount,total_collected,total_tax,total_gross
count,2.165105e+07,21651048.0,0.0,2.165105e+07,2.165105e+07,21651048.0,2.165105e+07,2.165105e+07,2.165105e+07
mean,7.013087e+08,740514.0,NaN,1.670496e+00,1.416418e+04,0.0,2.281375e+04,6.151670e+01,2.287526e+04
std,8.504971e+07,0.0,NaN,3.108776e+00,1.149599e+04,0.0,3.449838e+04,3.997916e+02,3.450744e+04
min,6.018361e+08,740514.0,NaN,0.000000e+00,1.000000e+03,0.0,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.283614e+08,740514.0,NaN,1.000000e+00,1.000000e+04,0.0,1.200000e+04,0.000000e+00,1.200000e+04
50%,6.521308e+08,740514.0,NaN,1.000000e+00,1.300000e+04,0.0,1.500000e+04,0.000000e+00,1.500000e+04
75%,8.003212e+08,740514.0,NaN,2.000000e+00,1.500000e+04,0.0,2.500000e+04,0.000000e+00,2.500000e+04
max,8.217413e+08,740514.0,NaN,5.700000e+02,2.705000e+06,0.0,2.850000e+06,1.610000e+04,2.850000e+06


In [87]:
# Check null values
df.isnull().sum()

id                        0
outlet_id                 0
settled_at                0
name                      0
sku                37343517
quantity                  0
unit_name                 0
price_sell_unit           0
payment_mode              0
total_discount            0
total_collected           0
total_tax                 0
total_gross               0
dtype: int64

In [88]:
df.dtypes

id                   int64
outlet_id            int64
settled_at          object
name                object
sku                float64
quantity           float64
unit_name           object
price_sell_unit    float64
payment_mode        object
total_discount     float64
total_collected    float64
total_tax          float64
total_gross        float64
dtype: object

In [89]:
df.columns

Index(['id', 'outlet_id', 'settled_at', 'name', 'sku', 'quantity', 'unit_name', 'price_sell_unit', 'payment_mode', 'total_discount', 'total_collected', 'total_tax', 'total_gross'], dtype='object')

In [117]:
# Convert datetime format:
df['settled_at'] = pd.to_datetime(df['settled_at'], format='%Y-%m-%d %H:%M:%S')

print(df['settled_at'].head())
print(df['settled_at'].dtype)

0   2024-02-29 19:13:12
1   2024-02-29 19:13:12
2   2024-02-29 19:13:12
3   2024-02-29 19:13:12
4   2024-02-29 19:12:15
Name: settled_at, dtype: datetime64[ns]
datetime64[ns]


In [118]:
# Setting index as the datetime column for easier manipulations:

y = df.set_index('settled_at')

In [119]:
y.index

DatetimeIndex(['2024-02-29 19:13:12', '2024-02-29 19:13:12', '2024-02-29 19:13:12', '2024-02-29 19:13:12', '2024-02-29 19:12:15', '2024-02-29 19:12:15', '2024-02-29 19:12:15', '2024-02-29 19:12:15', '2024-02-29 19:12:15', '2024-02-29 19:12:15',
               ...
               '2025-05-21 12:26:52', '2025-05-21 12:13:47', '2025-05-21 12:13:47', '2025-05-21 12:13:47', '2025-05-21 12:13:47', '2025-05-21 12:13:47', '2025-05-21 12:13:47', '2025-05-21 12:13:47', '2025-05-21 12:13:47', '2025-05-21 12:13:47'], dtype='datetime64[ns]', name='settled_at', length=21651048, freq=None)

In [120]:
# y['quantity'].plot(figsize=(15, 6))

# plt.title('Time Series Plot of Quantity')
# plt.xlabel('Date and Time') 
# plt.ylabel('Quantity')

# plt.grid(True) 
# plt.tight_layout() 
# plt.show()


In [121]:
# Drop column SKU
df = df.drop(columns=['sku'])

In [122]:
print(df.isnull().sum())

id                      0
outlet_id               0
settled_at              0
name               726831
quantity                0
unit_name          726831
price_sell_unit         0
payment_mode            0
total_discount          0
total_collected         0
total_tax               0
total_gross             0
dtype: int64


In [96]:
df.shape

(37343517, 12)

In [123]:
# (726831 / 21651048) * 100% ≈ 3.36% itu relatif kecil jdi lbh baik dibuang...
df_cleaned_name = df.dropna(subset=['name'])
print(df_cleaned_name.isnull().sum())

id                 0
outlet_id          0
settled_at         0
name               0
quantity           0
unit_name          0
price_sell_unit    0
payment_mode       0
total_discount     0
total_collected    0
total_tax          0
total_gross        0
dtype: int64


In [124]:
df['name'].nunique()

236

In [125]:
df_cleaned_name.dtypes

id                          int64
outlet_id                   int64
settled_at         datetime64[ns]
name                       object
quantity                  float64
unit_name                  object
price_sell_unit           float64
payment_mode               object
total_discount            float64
total_collected           float64
total_tax                 float64
total_gross               float64
dtype: object

In [126]:
df_cleaned_name[df_cleaned_name['name'] == 'Nasi Goreng Oriental']

,id,outlet_id,settled_at,name,quantity,unit_name,price_sell_unit,payment_mode,total_discount,total_collected,total_tax,total_gross
1,601908266,740514,2024-02-29 19:13:12,Nasi Goreng Oriental,1.0,Pcs,13000.0,CASH,0.0,13000.0,0.0,13000.0
13,601890655,740514,2024-02-29 18:48:27,Nasi Goreng Oriental,2.0,Pcs,13000.0,CASH,0.0,26000.0,0.0,26000.0
25,601888426,740514,2024-02-29 18:45:14,Nasi Goreng Oriental,2.0,Pcs,13000.0,CASH,0.0,26000.0,0.0,26000.0
44,601852024,740514,2024-02-29 17:50:00,Nasi Goreng Oriental,2.0,Pcs,13000.0,CASH,0.0,26000.0,0.0,26000.0
49,601850603,740514,2024-02-29 17:48:06,Nasi Goreng Oriental,1.0,Pcs,13000.0,CASH,0.0,13000.0,0.0,13000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21650830,821338152,740514,2025-05-20 16:24:02,Nasi Goreng Oriental,1.0,Pcs,15000.0,CASH,0.0,15000.0,0.0,15000.0
21650849,821320318,740514,2025-05-20 15:54:36,Nasi Goreng Oriental,1.0,Pcs,15000.0,CASH,0.0,15000.0,0.0,15000.0
21650865,821526016,740514,2025-05-20 21:44:41,Nasi Goreng Oriental,1.0,Pcs,15000.0,CASH,0.0,15000.0,0.0,15000.0
21650926,821521959,740514,2025-05-20 21:29:59,Nasi Goreng Oriental,2.0,Pcs,15000.0,CASH,0.0,30000.0,3000.0,33000.0


In [127]:
df_cleaned_name['settled_at'] = pd.to_datetime(df_cleaned_name['settled_at'], format='%Y-%m-%d %H:%M:%S')
df_cleaned_name = df_cleaned_name.set_index('settled_at')

In [128]:
df_cleaned_name.dtypes

id                   int64
outlet_id            int64
name                object
quantity           float64
unit_name           object
price_sell_unit    float64
payment_mode        object
total_discount     float64
total_collected    float64
total_tax          float64
total_gross        float64
dtype: object

In [131]:
# df_cleaned_name.loc['2024-02-29']
df_feb29_filtered = df_cleaned_name.loc['2024-02-29']
df_filtered_by_date_and_name = df_feb29_filtered[df_feb29_filtered['name'] == 'Air mineral']
total_quantity_air_mineral = df_filtered_by_date_and_name['quantity'].sum()

print(total_quantity_air_mineral)

1245.0


In [103]:
daily_aggregated_sales = df_cleaned_name.groupby(['name', pd.Grouper(freq='D')]).agg(
    quantity=('quantity', 'sum'),
    # Ambil harga jual per unit TERTINGGI untuk hari itu
    price_sell_unit=('price_sell_unit', 'max'),
    total_discount=('total_discount', 'sum'),
    total_collected=('total_collected', 'sum'),
    total_tax=('total_tax', 'sum'),
    total_gross=('total_gross', 'sum')
).reset_index() # Mengembalikan 'name' dan 'settled_at' menjadi kolom biasa

# Mengganti nama kolom 'settled_at' menjadi 'date' untuk kejelasan
daily_aggregated_sales.rename(columns={'settled_at': 'date'}, inplace=True)

In [104]:
daily_aggregated_sales.head()

,name,date,quantity,price_sell_unit,total_discount,total_collected,total_tax,total_gross
0,Kuah Sate,2025-04-22,2118.0,6000.0,0.0,12708000.0,0.0,12708000.0
1,*AIR HANGAT - Reguler,2023-06-30,1059.0,2000.0,0.0,2118000.0,0.0,2118000.0
2,*AIR HANGAT - Reguler,2023-07-28,5295.0,2000.0,0.0,10590000.0,0.0,10590000.0
3,*AIR HANGAT - Reguler,2023-12-12,1059.0,2000.0,0.0,2118000.0,0.0,2118000.0
4,*AIR HANGAT - Reguler,2023-12-14,1059.0,2000.0,0.0,2118000.0,0.0,2118000.0


In [105]:
# Tanggal Minimum
min_date = daily_aggregated_sales['date'].min()

# Tanggal Maksimum
max_date = daily_aggregated_sales['date'].max()

print(f"Tanggal Minimum: {min_date}")
print(f"Tanggal Maksimum: {max_date}")

Tanggal Minimum: 2023-06-23 00:00:00
Tanggal Maksimum: 2025-05-21 00:00:00


In [106]:
# --- 3. Terapkan Pivot Table Langsung pada daily_aggregated_sales ---

# Gunakan pivot_table untuk mengubah 'name' menjadi kolom, dengan 'quantity' sebagai nilainya
# 'date' akan menjadi indeks baru
df_pivoted_quantity = daily_aggregated_sales.pivot_table(
    index='date',
    columns='name',
    values='quantity',
    fill_value=0 # Mengisi nilai 0 jika ada tanggal di mana produk lain terjual tapi produk ini tidak
)

In [107]:
df_pivoted_quantity.to_csv('penjualan_produk_harian.csv', index=True)

In [108]:
print(df_pivoted_quantity.head(10))

name         Kuah Sate   *AIR HANGAT - Reguler  *AIR MINERAL - Reguler  *AYAM BALADO NO PAKET  *AYAM PETEI BALADO NO PAKET  *BAKSO - Reguler  *BAKSO - gofood  *BIGOR CABE IJO - Reguler  *BIGOR CABE IJO - gofood  *BIGOR IKAN ASIN - gofood  *BIGOR KAMPUNG - Reguler  *BIGOR KAMPUNG - gofood  *BIGOR LADA HITAM - Reguler  *BIGOR NANAS - Reguler  *BIGOR NANAS - gofood  *BIGOR PETEI - Reguler  *BIGOR PETEI - gofood  *BIGOR RENDANG - Reguler  *BIGOR SEAFOOD - Gofood  *BIGOR SEAFOOD - Reguler  *BIGOR SOSIS - Gofood  *BIGOR SOSIS - Reguler  *BIGOR TOMYAM  - Reguler  *BIGOR TOMYAM  - gofood  *BIGOR XIANGCHUN - Reguler  *BIGOR XIANGCHUN - gofood  *BIHUN SIRAM - Gofoid  *BIHUN SIRAM - Reguler  *BUBUR AYAM - Reguler  *BUBUR AYAM - gofood  *BUBUR SAYUR - Reguler  *BUBUR SAYUR - gofood  *BURGER MINI - Reguler  *BURGER MINI - gofood  *Bakcang - Gofood  *Bakcang - Reguler  *CAP CAI NO PAKET - Reguler  *CAP CAI NO PAKET - gofood  *CHAI KWE/SUN KUE - Reguler  *CHAI KWE/SUN KUE - gofood  *CHE HUN TIAU - Re